In [33]:
import pandas as pd
import math
from decimal import *
import numpy as np

In [34]:
dfAden = pd.read_csv('../data/aden.csv',sep=';')

# Affectation des zones

In [35]:
min_lat=-10
max_lat=31
min_lon=31
max_lon=78

#un degre de latitude "vaut"  111,11 km
accuracy_km=65
accuracy_lat=accuracy_km/111.11

diff_lat=max_lat-min_lat
diff_lon=max_lon-min_lon

path_lat=float(diff_lat)/float(accuracy_lat)

latitude_grid=np.linspace(min_lat, max_lat, path_lat)
longitude_grid=[]
longitude_grid.append(min_lon)

tab=[]
for i in range(0,len(latitude_grid)-1):
    mean_lat=float(latitude_grid[i]+latitude_grid[i+1])/2
    
    #print mean_lat
    #print math.cos(math.radians(mean_lat))
    km_lon=111.11*math.cos(math.radians(mean_lat))
    accuracy_lon=accuracy_km/km_lon
    
    tab.append(accuracy_lon)
    
    currentLon=longitude_grid[i]+accuracy_lon
    longitude_grid.append(currentLon)
    #print 111.11*math.cos(math.radians(mean_lat))

    
#on complete jusqu'a 78
maxCurrent=max(longitude_grid)
meanPath=np.mean(tab)
while maxCurrent<78:
    maxCurrent=max(longitude_grid)
    longitude_grid.append(maxCurrent+meanPath)
    
    
print len(latitude_grid)
print len(longitude_grid)
#print latitude_grid
#print longitude_grid


70
80


In [36]:
def affectation_zone(latitude,longitude,latitude_grid,longitude_grid,lat_bool):
    #print 'latitude : '+ str(latitude)
    #print 'longitude : '+ str(longitude)
    
    for i in range(0,len(latitude_grid)-1):
        if latitude<latitude_grid[i+1] and latitude>latitude_grid[i]:
            idx_zone_lat=i
            
    for i in range(0,len(longitude_grid)-1):
        if longitude<longitude_grid[i+1] and longitude>longitude_grid[i]:
            idx_zone_lon=i

    #print idx_zone_lon
    #print idx_zone_lat
    if lat_bool:
        return idx_zone_lat
    else:
        return idx_zone_lon

In [37]:
latitude=9.91666666667
longitude=76.0833333333
idx_zone_lat=affectation_zone(latitude,longitude,latitude_grid,longitude_grid,True)
idx_zone_lon=affectation_zone(latitude,longitude,latitude_grid,longitude_grid,False)
print idx_zone_lat
print idx_zone_lon

33
74


In [38]:
dfAden['lat_area'] = dfAden.apply(lambda row: affectation_zone(row['latitude'],row['longitude'],latitude_grid,longitude_grid,True), axis=1)
dfAden['lon_area'] = dfAden.apply(lambda row: affectation_zone(row['latitude'],row['longitude'],latitude_grid,longitude_grid,False), axis=1)

In [43]:
dfAden['Annee (0)'] = dfAden['Annee (0)'].astype(int)
dfAden['jour (0)'] = dfAden['jour (0)'].astype(int)
dfAden['mois (0)'] = dfAden['mois (0)'].astype(int)

In [59]:
#dfAden.columns[0]
columnTodelete=filter(lambda x:'Unna' in x, dfAden.columns)

print columnTodelete
for col in columnTodelete:
    print col
    dfAden = dfAden.drop(col)

['Unnamed: 0.1.1']
Unnamed: 0.1.1


ValueError: labels ['Unnamed: 0.1.1'] not contained in axis

In [60]:
dfAden.to_csv("../data/aden.csv",sep=';', encoding='utf-8')

In [58]:
dfAden.columns

Index([u'Unnamed: 0.1.1', u'Annee (0)', u'jour (0)', u'mois (0)',
       u'dist à la côte (nq)', u'zone', u'lat (0)', u'long(0)', u'type(0)',
       u'pavillon', u'statut de l'attaque(0)', u'type d'attaque(0)',
       u'Nombre P  Min', u'Nombre P  max', u'nombre embarcation',
       u'Vol à bord', u' Kidnapping', u'blessées', u' meurtre', u'Knives ',
       u'RPG', u'descriptif', u'latitude', u'longitude', u'quinzaine_week',
       u'newDescriptif', u'Guns', u'mousson', u'distance au port',
       u'distance_shorline', u'lat_area', u'lon_area'],
      dtype='object')